# Межквартильный диапазон (IQR)

1. **Рассчитайте IQR:** IQR — это разница между 75-м (Q3) и 25-м (Q1) процентилями набора данных.

   ```
   IQR = Q3 - Q1
   ```

2. **Определите границы:** Верхняя и нижняя границы определяются следующим образом:

   ```
   Верхняя граница = Q3 + 1.5 * IQR
   Нижняя граница = Q1 - 1.5 * IQR
   ```

3. **Выявите выбросы:** Любое значение данных выше верхней границы или ниже нижней границы считается выбросом.

In [3]:
import numpy as np
import pandas as pd

# Пример данных
data = {'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100]}
df = pd.DataFrame(data)

# IQR
Q1 = df['value'].quantile(0.25)
Q3 = df['value'].quantile(0.75)
IQR = Q3 - Q1


mx = Q3 + 1.5 * IQR
mn = Q1 - 1.5 * IQR


filtered_df = df[(df['value'] >= mn) & (df['value'] <= mx)]

filtered_df


,value
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


## Метод 2: Z-оценка

1. **Рассчитайте z-оценку:** Z-оценка для каждого значения данных рассчитывается как:

   ```
   z = (x - среднее) / стандартное отклонение
   ```
   где `x` — значение данных, `среднее` — среднее значение набора данных, а `стандартное отклонение` — стандартное отклонение набора данных.

2. **Выявите выбросы:** Выбросы обычно определяются как значения со z-оценкой больше 3 или меньше -3. Это основано на правиле трех сигм, которое гласит, что около 99,7% значений данных в нормальном распределении находятся в пределах трех стандартных отклонений от среднего значения.

### Обработка выбросов

* **Удаление:**  Этот метод включает в себя полное удаление выбросов из набора данных.  Он прост в реализации, но может привести к потере данных, особенно если выбросов много или они представляют собой важные закономерности.

* **Преобразование:**  Методы преобразования, такие как логарифмическое, квадратного корня или преобразование Бокса-Кокса, могут помочь уменьшить влияние выбросов, сделав данные более нормальными.  Это особенно полезно, когда выбросы обусловлены искаженным распределением.

* **Приписывание:**  Выбросы могут быть приписаны (заменены) другими значениями, такими как среднее, медиана или значение, предсказанное моделью.  Это сохраняет все точки данных, но может вводить смещения, если приписываемые значения неточно отражают истинные значения.  Этот метод может быть особенно полезен, когда удаление данных нежелательно.

* **Winsorizing:** Winsorizing предполагает ограничение значений выбросов до определенного процентиля. Например, значения выше 95-го процентиля могут быть установлены равными значению 95-го процентиля, а значения ниже 5-го процентиля могут быть установлены равными значению 5-го процентиля. Это помогает уменьшить влияние выбросов, не полностью удаляя их.

* **Отдельный анализ:**  Иногда выбросы могут представлять собой важные закономерности или подгруппы.  В таких случаях их можно проанализировать отдельно от остальной части данных, чтобы получить более полное представление о наборе данных.


### Выбор подходящей стратегии:

Лучшая стратегия обработки выбросов зависит от конкретного набора данных и целей анализа.  Учитывайте следующие факторы:

* **Количество выбросов:**  Если выбросов мало, удаление может быть приемлемым.  Однако, если выбросов много, удаление может привести к значительной потере данных.

* **Причина выбросов:**  Если выбросы обусловлены ошибками измерения или ввода данных, удаление может быть лучшим вариантом.  Однако, если выбросы представляют собой подлинные значения данных, преобразование или приписывание может быть более подходящим.

* **Влияние на анализ:**  Учитывайте влияние выбросов на анализ.  Если выбросы существенно влияют на результаты, возможно, потребуется их обработать.

In [ ]:
import numpy as np
import pandas as pd

# Пример данных
data = {'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 110]}
df = pd.DataFrame(data)

# Рассчитать Z-оценку
df['z_score'] = (df['value'] - df['value'].mean()) / df['value'].std()

# Выявить выбросы (порог = 3)
outliers = df[(df['z_score'] > 3) | (df['z_score'] < -3)]

# Фильтр выбросов
filtered_df = df[(df['z_score'] <= 3) & (df['z_score'] >= -3)]

print("DataFrame с Z-оценкой:")
print(df)
print("\nВыбросы:")
print(outliers)
print("\nDataFrame без выбросов:")
print(filtered_df)



# Приписывание выбросам среднего значения
df_imputed = df.copy()
mean_value = df_imputed[(df_imputed['z_score'] <= 3) & (df_imputed['z_score'] >= -3)]['value'].mean()

df_imputed.loc[(df_imputed['z_score'] > 3) | (df_imputed['z_score'] < -3), 'value'] = mean_value
df_imputed['z_score'] = (df_imputed['value'] - df_imputed['value'].mean()) / df_imputed['value'].std()

print("\nDataFrame с приписанными выбросами:")
print(df_imputed)

DataFrame с Z-оценкой:
    value   z_score
0       1 -0.442498
1       2 -0.410891
2       3 -0.379284
3       4 -0.347677
4       5 -0.316070
5       6 -0.284463
6       7 -0.252856
7       8 -0.221249
8       9 -0.189642
9      10 -0.158035
10    110  3.002663

Выбросы:
    value   z_score
10    110  3.002663

DataFrame без выбросов:
   value   z_score
0      1 -0.442498
1      2 -0.410891
2      3 -0.379284
3      4 -0.347677
4      5 -0.316070
5      6 -0.284463
6      7 -0.252856
7      8 -0.221249
8      9 -0.189642
9     10 -0.158035

DataFrame с приписанными выбросами:
    value   z_score
0     1.0 -1.566699
1     2.0 -1.218544
2     3.0 -0.870388
3     4.0 -0.522233
4     5.0 -0.174078
5     6.0  0.174078
6     7.0  0.522233
7     8.0  0.870388
8     9.0  1.218544
9    10.0  1.566699
10    5.5  0.000000


## Одномерные распределения

Одномерные распределения — мощный инструмент для выявления аномалий и ошибок в данных.  Они позволяют увидеть, как значения каждой отдельной переменной распределены, что может помочь обнаружить несоответствия или необычные закономерности, указывающие на ошибки или шум. Вот как их можно использовать для очистки данных:

**1. Визуализация:**

* **Гистограммы:** Гистограммы показывают частоту значений в определенных интервалах (бинаx). Они отлично подходят для выявления:
    * **Выбросов:**  Значения, которые сильно отличаются от основного распределения, могут быть выбросами (ошибками или шумом).
    * **Невозможные значения:**  Например, отрицательный возраст или доход.
    * **Необычные пики или провалы:**  Могут указывать на ошибки ввода данных или систематические проблемы.
    * **Асимметрия:**  Асимметричное распределение может указывать на необходимость преобразования данных.

* **Ящичковые диаграммы (Box Plots):** Box plots показывают медиану, квартили и выбросы. Они полезны для:
    * **Выявления выбросов:**  Значения за пределами "усов" ящика обычно считаются выбросами.
    * **Сравнения распределений:**  Box plots позволяют легко сравнить распределения разных групп или переменных.

* **Диаграммы плотности (Density Plots):**  Density plots показывают гладкую оценку распределения.  Они могут быть полезны для:
    * **Выявления мультимодальности:**  Наличие нескольких пиков может указывать на то, что данные состоят из нескольких различных групп.
    * **Выявления асимметрии:**  Форма кривой плотности может показать, насколько асимметрично распределение.

**2. Описательная статистика:**

* **Среднее, медиана, мода:**  Сравнение этих мер центральной тенденции может выявить асимметрию.  Например, если среднее значение значительно больше медианы, это может указывать на наличие выбросов, тянущих среднее значение вверх.
* **Стандартное отклонение, диапазон, IQR:**  Эти меры рассеяния показывают, насколько разбросаны данные.  Необычно большие или малые значения могут указывать на ошибки или шум.
* **Процентили:**  Проверка определенных процентилей (например, 1-й, 5-й, 95-й, 99-й) может помочь выявить экстремальные значения, которые могут быть ошибками.
* **Квантили:**  Аналогично процентилям, квантили могут использоваться для определения границ для выявления выбросов.
